# Text-to-Image Generation with Laxhar/noobai-XL-1.1

This notebook allows you to utilize the `Laxhar/noobai-XL-1.1` text-to-image model on Google Colab. You can customize various parameters such as sampler methods, sampling steps, seeds, CFG scale, image size, and apply a high-resolution fix.

## Table of Contents
1. [Setup Environment](#1-Setup-Environment)
2. [Import Libraries](#2-Import-Libraries)
3. [Configure Device](#3-Configure-Device)
4. [Modify Tokenizer Configuration](#4-Modify-Tokenizer-Configuration)
5. [Load the Model](#5-Load-the-Model)
6. [Define Helper Functions](#6-Define-Helper-Functions)
7. [High-Resolution Fix (8x-NKMD-Superscale)](#7-High-Resolution-Fix-8x-NKMD-Superscale)
8. [User Inputs](#8-User-Inputs)
9. [Generate and Display Image](#9-Generate-and-Display-Image)
10. [Save the Image](#10-Save-the-Image)
11. [Summary](#11-Summary)
12. [Additional Resources](#12-Additional-Resources)

---

## 1. Setup Environment

First, install the necessary libraries required for running the model and processing images.

In [ ]:
# Install necessary libraries
!pip install diffusers transformers accelerate scipy safetensors

# Install additional libraries for high-resolution fixes
!pip install k-diffusion

---

## 2. Import Libraries

Import all the necessary libraries for model loading, image processing, and setting configurations.

In [ ]:
import torch
from diffusers import StableDiffusionXLPipeline, EulerDiscreteScheduler
from transformers import CLIPTokenizer
import random
import numpy as np
from PIL import Image
import json
import k_diffusion as K
from k_diffusion import sampling

---

## 3. Configure Device

Check if a GPU is available and set the device accordingly. GPUs significantly speed up model inference.

In [ ]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

---

## 4. Modify Tokenizer Configuration

To remove the character limit, we'll modify the tokenizer configurations by removing the `max_length` parameter.

In [ ]:
# Load the tokenizer
tokenizer = CLIPTokenizer.from_pretrained("Laxhar/noobai-XL-1.1", use_fast=True)

# Function to remove max_length from tokenizer config
def remove_max_length(config_path):
    with open(config_path, 'r') as f:
        config = json.load(f)
    if 'max_length' in config:
        del config['max_length']
    with open(config_path, 'w') as f:
        json.dump(config, f, indent=4)
    print(f"Removed 'max_length' from {config_path}")

# Paths to tokenizer configurations
tokenizer_config_path = "./tokenizer/tokenizer_config.json"
tokenizer_2_config_path = "./tokenizer_2/tokenizer_config.json"

# Remove max_length from both tokenizer configs
remove_max_length(tokenizer_config_path)
remove_max_length(tokenizer_2_config_path)

---

## 5. Load the Model

Load the scheduler and the Stable Diffusion XL Pipeline model. Enable memory-efficient attention for better performance.

In [ ]:
# Load the scheduler
scheduler = EulerDiscreteScheduler.from_config("Laxhar/noobai-XL-1.1")

# Load the pipeline
pipe = StableDiffusionXLPipeline.from_pretrained(
    "Laxhar/noobai-XL-1.1",
    scheduler=scheduler,
    torch_dtype=torch.float16
).to(device)

# Enable memory-efficient attention if available
try:
    pipe.enable_xformers_memory_efficient_attention()
    print("Enabled memory-efficient attention.")
except Exception as e:
    print(f"Could not enable memory-efficient attention: {e}")

---

## 6. Define Helper Functions

These functions will help set seeds for reproducibility and generate images based on user inputs.

In [ ]:
def set_seed(seed):
    """
    Sets the seed for reproducibility.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if device == "cuda":
        torch.cuda.manual_seed_all(seed)

def generate_image(prompt, sampler="euler", steps=50, seed=42, cfg_scale=7.5, image_size=(512, 512)):
    """
    Generates an image based on the prompt and parameters.
    """
    set_seed(seed)
    
    # Update scheduler if sampler is changed
    if sampler.lower() == "euler":
        pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)
    # Add other samplers here if needed
    
    # Generate image
    image = pipe(
        prompt,
        num_inference_steps=steps,
        guidance_scale=cfg_scale,
        height=image_size[0],
        width=image_size[1]
    ).images[0]
    
    return image

---

## 7. High-Resolution Fix (8x-NKMD-Superscale)

Apply a high-resolution fix to upscale the generated image. This implementation uses a placeholder for the `8x-NKMD-Superscale` method. Replace the placeholder with the actual superscaling method as needed.

In [ ]:
def hires_fix(image, scale=2.0, steps=50, seed=123):
    """
    Applies a high-resolution fix to the image.
    """
    set_seed(seed)
    
    # Placeholder for actual 8x-NKMD-Superscale implementation
    # Replace the following code with the actual superscaling method
    
    # Example: Simple upscaling using PIL (replace with NKMD-Superscale)
    upscaled_image = image.resize((int(image.width * scale), int(image.height * scale)), Image.LANCZOS)
    
    return upscaled_image

---

## 8. User Inputs

Define the parameters for image generation. You can modify these values to customize the output.

In [ ]:
# Define user inputs
prompt = "A futuristic cityscape at sunset with flying cars"
sampler = "euler"              # Options: 'euler', 'other_samplers'
steps = 50                     # Sampling steps
seed = 42                      # Random seed for reproducibility
cfg_scale = 7.5                # CFG scale for guidance
image_size = (512, 512)        # Image size (height, width)

# High-resolution fix parameters
apply_hires = True             # Set to True to apply high-resolution fix
scale = 2.0                    # Upscaling factor
hires_steps = 50               # Steps for high-resolution fix
hires_seed = 123               # Seed for high-resolution fix

---

## 9. Generate and Display Image

Generate the image based on the defined parameters and display it.

In [ ]:
# Generate image
image = generate_image(prompt, sampler, steps, seed, cfg_scale, image_size)

# Apply high-resolution fix if enabled
if apply_hires:
    image = hires_fix(image, scale, hires_steps, hires_seed)

# Display the image
image.show()

---

## 10. Save the Image

Save the generated image to the local file system.

In [ ]:
# Save the generated image
image_path = "generated_image.png"
image.save(image_path)
print(f"Image saved to {image_path}")

---

## 11. Summary

- **Sampler Methods:** You can choose different sampler methods by modifying the `sampler` variable.
- **Sampling Steps:** Adjust the `steps` variable to change the number of inference steps.
- **Seeds:** Set the `seed` variable for reproducible results.
- **CFG Scale:** Modify the `cfg_scale` variable to control the guidance scale.
- **Image Size:** Change the `image_size` tuple to generate images of different dimensions.
- **High-Resolution Fix:** Enable or disable high-resolution upscaling and adjust its parameters accordingly.

---

## 12. Additional Resources

- [Hugging Face Diffusers Documentation](https://huggingface.co/docs/diffusers/index)
- [Stable Diffusion Documentation](https://stability.ai/blog/stable-diffusion-public-release)
- [K-Diffusion GitHub Repository](https://github.com/crowsonkb/k-diffusion)

---

**Note:** Ensure you have the necessary permissions and comply with the model's license terms when using and modifying it.